In [1]:
# שליפת תוכן כותרות והיררכיות, כולל בדיקת סיום טקסט התוכן לפי סיומות המעידות על התחלת הפניות ואזכורי עדכונים ביחס לגרסאות ישנות של קוד המס

In [9]:
import pandas as pd
from docx import Document

# Load the Excel file containing the hierarchy levels and titles
excel_file = "D:\\yifat\\Extract_Data_Project\\Headlines 2018.xlsx"
df = pd.read_excel(excel_file)

# Extract the relevant columns from the Excel file
titles = df['hierarchy_level_name'].tolist()

# Load the Word document
word_file = "D:\\yifat\\Extract_Data_Project\\TITLE 26_2018_new2.docx"
doc = Document(word_file)

# Define the strings that indicate stopping points
stop_strings = ["(Aug. 16, 1954", "(Added Pub. L."]

# Initialize variables to store extracted data
content_list = []
current_hierarchy = None
current_content = []

# Function to save the current content when switching to a new hierarchy
def save_content():
    if current_hierarchy and current_content:
        content_text = "\n".join(current_content).strip()
        content_list.append({
            'hierarchy_level_name': current_hierarchy,
            'content': content_text,
            'word_count': len(content_text.split())
        })

# Iterate through the paragraphs in the Word document
for para in doc.paragraphs:
    para_text = para.text.strip()
    if para_text in titles:
        # Save the current hierarchy content if switching to a new one
        save_content()
        current_hierarchy = para_text
        current_content = []
    elif current_hierarchy:
        # Stop appending content if we reach a stop string
        if any(stop_string in para_text for stop_string in stop_strings):
            save_content()
            current_hierarchy = None
            current_content = []
        else:
            # Append the paragraph to the current hierarchy content
            current_content.append(para_text)

# Save the last hierarchy content
save_content()

# Convert the extracted data to a DataFrame
output_df = pd.DataFrame(content_list)

# Add the original Excel columns to the DataFrame
output_df = df.merge(output_df, on='hierarchy_level_name', how='left')

# Save the output to a new Excel file
output_file = "D:\\yifat\\Extract_Data_Project\\Extracted_Hierarchy_Content_2018.xlsx"
output_df.to_excel(output_file, index=False)

print(f"Extracted content saved to {output_file}")


Extracted content saved to D:\yifat\Extract_Data_Project\Extracted_Hierarchy_Content_2018.xlsx
